In [8]:
import kagglehub

dataset_path = kagglehub.dataset_download("andrewmvd/animal-faces")
print("Dataset path:", dataset_path)

Dataset path: /Users/keerthanamanoharan/.cache/kagglehub/datasets/andrewmvd/animal-faces/versions/1


In [9]:
print(dataset_path)

/Users/keerthanamanoharan/.cache/kagglehub/datasets/andrewmvd/animal-faces/versions/1


## qqwe

In [19]:
import tensorflow as tf
import os

# Dataset path (replace this with the real path if needed)
data_dir = os.path.expanduser(dataset_path)

# Load dataset from directory
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Class Name
class_names = train_ds.class_names
num_classes = len(class_names)
print("class_names")


Found 16130 files belonging to 1 classes.
Using 12904 files for training.
Found 16130 files belonging to 1 classes.
Using 3226 files for validation.
class_names


In [20]:
# Normalize pixel values to [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))


In [18]:
#Build and Train the CNN Model
num_classes = len(train_ds.class_names)  # e.g., 3 for cat, dog, panda

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history = model.fit(train_ds, validation_data=val_ds, epochs=10)


Epoch 1/10


/Users/keerthanamanoharan/Desktop/FromClass/Repos/identify-animal-in-pics/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


404/404 ━━━━━━━━━━━━━━━━━━━━ 147s 363ms/step - accuracy: 0.1004 - loss: 0.0000e+00 - val_accuracy: 0.1135 - val_loss: 0.0000e+00
Epoch 2/10
404/404 ━━━━━━━━━━━━━━━━━━━━ 149s 369ms/step - accuracy: 0.1009 - loss: 0.0000e+00 - val_accuracy: 0.1135 - val_loss: 0.0000e+00
Epoch 3/10
404/404 ━━━━━━━━━━━━━━━━━━━━ 148s 366ms/step - accuracy: 0.1012 - loss: 0.0000e+00 - val_accuracy: 0.1135 - val_loss: 0.0000e+00
Epoch 4/10
404/404 ━━━━━━━━━━━━━━━━━━━━ 151s 373ms/step - accuracy: 0.1004 - loss: 0.0000e+00 - val_accuracy: 0.1135 - val_loss: 0.0000e+00
Epoch 5/10
404/404 ━━━━━━━━━━━━━━━━━━━━ 149s 368ms/step - accuracy: 0.1012 - loss: 0.0000e+00 - val_accuracy: 0.1135 - val_loss: 0.0000e+00
Epoch 6/10
404/404 ━━━━━━━━━━━━━━━━━━━━ 149s 370ms/step - accuracy: 0.1013 - loss: 0.0000e+00 - val_accuracy: 0.1135 - val_loss: 0.0000e+00
Epoch 7/10
404/404 ━━━━━━━━━━━━━━━━━━━━ 151s 375ms/step - accuracy: 0.1012 - loss: 0.0000e+00 - val_accuracy: 0.1135 - val_loss: 0.0000e+00
Epoch 8/10
404/404 ━━━━━━━━━━━━

In [21]:
#Evaluate the Model
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy*100:.2f}%")


101/101 ━━━━━━━━━━━━━━━━━━━━ 10s 99ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Validation Accuracy: 100.00%


In [24]:
#Analyze and Reflect
from sklearn.metrics import classification_report
import numpy as np

# True labels
y_true = np.concatenate([y.numpy() for x, y in val_ds])
# Predictions
y_pred_logits = model.predict(val_ds)
y_pred = np.argmax(y_pred_logits, axis=1)

# Print report  
print(classification_report(y_true, y_pred, target_names=class_names))


101/101 ━━━━━━━━━━━━━━━━━━━━ 10s 95ms/step
              precision    recall  f1-score   support

        afhq       1.00      1.00      1.00      3226

    accuracy                           1.00      3226
   macro avg       1.00      1.00      1.00      3226
weighted avg       1.00      1.00      1.00      3226



Dataset: Animal Faces from Kaggle (cats, dogs, pandas)

Model: CNN trained from scratch using Keras

Accuracy: Reported on validation set + detailed classification metrics

Improvements:

Add data augmentation

Use transfer learning with MobileNet or ResNet

Hyperparameter tuning (more epochs, different optimizers, etc.)

Would you like me to show this using transfer learning (for higher accuracy), or do you want to visualize predictions with images?